1. Completar la tabla

In [2]:
import torch
import torchsummary

In [3]:
class nnTable(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels= 3, out_channels= 8, kernel_size=5, stride=2, padding= 2)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding= 1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=4, stride=4)
        self.fc1 = torch.nn.Linear(in_features= 16 * 4 * 4, out_features=6)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = self.fc1(x)
        return x

conv_model = nnTable()

In [4]:
torchsummary.summary(conv_model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             608
         MaxPool2d-2            [-1, 8, 16, 16]               0
            Conv2d-3           [-1, 16, 16, 16]           1,168
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                    [-1, 6]           1,542
Total params: 3,318
Trainable params: 3,318
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.11
Params size (MB): 0.01
Estimated Total Size (MB): 0.17
----------------------------------------------------------------


|  | Dimensiones de entrada | Cantidad de parámetros |
| --- | --- | --- |
| Entrada   | (3, 64, 64) | 0 |
| Conv2D(f=5, s=2, c=8, p=2) | (8, 32, 32) | 608 |
| MaxPool(f=2, s=2) | (8, 16, 16) | 0 |
| Conv2D(f=3, s=1, c=16, p=1) | (16, 16, 16) | 1168 |
| MaxPool(f=4, s=4) | (16, 4, 4) | 0 |
| Dense(salida=6) | (6)| 1542 |

2) 

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [114]:
class Conv2d(object):
    
    def __init__(self, in_channels: int, out_channels: int, filter_size, stride, padding):

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        #self.w = np.random.randn(out_channels, in_channels, filter_size, filter_size)
        #self.b = np.random.randn(out_channels)
        #self.last_output = None
        #self.last_input = None
        
       
    # Padding

    def zero_padding(self, X):
      """
      Argumentos:
      X: Array numpy de entrada con dimensiones (batch_size, n_C, n_H, n_W)
      #pad: Entero representando la cantidad de filas y columnas que se deben agregar con ceros

      Retorna:

      X_padded: Array numpy con dimensiones (batch_size, n_C, n_H + 2*pad, n_W + 2*pad)
      """
      if self.padding.lower() == "valid":

          return X

      elif self.padding.lower() == "same":

          pad = ((self.stride -1) * X.shape[0] - self.stride + self.filter_size) // 2
          X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), 'constant', constant_values = (0,0))

          return X_padded, pad

    # Función para realizar la operación de convolución

    def convolve(self, X, W, b):
      """
      Argumentos:
        X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
        #W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
        #b: Entero con el valor de bias de la capa actual

      Retorna:
        Z: Entero con el valor del resultado
      """

      # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
      aux = X * W
      # Realizo la suma de todos los elementos
      aux = np.sum(aux)
      # Le sumo el valor del bias para obtener Z
      Z = aux + float(b)

      return Z
    
    # Función para realizar el forward pass de una capa convolucional

    # Función para realizar el forward pass de una capa convolucional

    def conv_forward(self, layer_input, W, b):
      """
      Argumentos:
        layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
        W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
        b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
        stride: Entero con el valor de stride utilizado en la capa actual.
        padding: 

      Retorna:
        Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
      """

      # Obtengo las dimensiones de la entrada
      (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape

      # Obtengo las dimensiones de los filtros
      (n_C, n_C_prev, filter_size, filter_size) = W.shape

      # Agrego padding con ceros al volumen de entrada
      layer_input_padded, pad = self.zero_padding(layer_input)

      # Calculo las dimensiones del volumen de salida de la capa actual
      n_H = int((n_H_prev + 2*pad - filter_size)/self.stride + 1)
      n_W = int((n_W_prev + 2*pad - filter_size)/self.stride + 1)

      # Inicializo el volumen de salida con ceros
      Z = np.zeros([batch_size, n_C, n_H, n_W])


      # Comienzo iterando sobre cada ejemplo del batch
      for i in range(batch_size):

        # Itero sobre el eje vertical del volumen de salida
        for h in range(n_H):
          # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          y_start = self.stride * h
          y_end = y_start + filter_size

          # Itero sobre el eje horizontal del volumen de salida
          for w in range(n_W):
            # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
            x_start = self.stride * w
            x_end = x_start + filter_size

            # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
            slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
            
            # Itero sobre la cantidad de canales del volumen de salida
            for c in range(n_C):

              # Obtengo el valor del filtro y bias del canal correspondiente
              filter = W[c, :, :, :]
              bias = b[c]

              # Computo la operación de convolución para esta ventana
              Z[i, c, h, w] = self.convolve(slice_from_input_padded, filter, bias)
      
      return Z


def conv_backward(self, dLdO, layer_input, W, b):

    #dimensiones de la derivada de L con respecto de O
    (batch_size, n_C, n_H, n_W) = dLdO.shape 
    
    #dimensiones de la entrada de la capa (output del forward)
        
    layer_output = self.conv_forward(layer_input, W, b)
    
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_output.shape

    # Obtengo las dimensiones de los filtros
    (n_C, n_C_prev, filter_size, filter_size) = W.shape

    
    # Inicializa dA_prev, dW, db 
    dA_prev = np.zeros((batch_size, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((filter_size, filter_size, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    # Pad A_prev y dA_prev
    A_prev_pad = self.zero_padding(layer_input)
    dA_prev_pad, pad = self.zero_padding(dA_prev)
    
    for i in range(batch_size):                       
        
        a_prev_pad = A_prev_pad[i, :, :, :]
        da_prev_pad = dA_prev_pad[i, :, :, :]
        
        for h in range(n_H):                   
            for w in range(n_W):             
                for c in range(n_C):           
                    
                    vert_start = h
                    vert_end = vert_start + filter_size
                    horiz_start = w
                    horiz_end = horiz_start + filter_size
                    
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dLdO[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dLdO[i, h, w, c]
                    db[:,:,:,c] += dLdO[i, h, w, c]
                    
        dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]
    
    return dW, db
   

### Validación resultados

In [108]:
# Dimensiones de la entrada
batch_size = 1
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 1
pad = "same"

# Variables de prueba
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)
W = np.random.randn(filters, input_channels, filter_size, filter_size)
b = np.random.randn(filters)


In [109]:
conv = Conv2d(input_channels, filters, filter_size, stride, pad)


#### Zero Padding

In [110]:
x_padded, pad = conv.zero_padding(test_array)
x_padded.shape


(1, 4, 9, 9)

#### Conv Forward

In [112]:
conv_result = conv.conv_forward(test_array, W, b)
conv_result_pyt = torch.nn.functional.conv2d(torch.tensor(test_array), torch.tensor(W), torch.tensor(b), stride, pad)


In [113]:
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[0, 0, 0, 0]))
print("Convolución: Result shape: {}".format(conv_result_pyt.shape))
print("Convolución: Result value: {}".format(conv_result_pyt[0, 0, 0, 0]))

Convolución: Result shape: (1, 8, 7, 7)
Convolución: Result value: 2.665044292399455
Convolución: Result shape: torch.Size([1, 8, 7, 7])
Convolución: Result value: 2.665044292399455


#### Conv Backward